In [3]:
import pandas as pd
import numpy as np

In [4]:
df_train = pd.read_csv('train.csv')
df_train.name = 'train'

df_oil = pd.read_csv('oil.csv')
df_oil.name = 'oil'

df_stores = pd.read_csv('stores.csv')
df_stores.name = 'stores'

df_transactions = pd.read_csv('transactions.csv')
df_transactions.name = 'transactions'

df_holidays = pd.read_csv('holidays_events.csv')
df_holidays.name = 'holidays'

dataframes = [df_train, df_oil, df_stores, df_transactions, df_holidays]

# Description

The dataset contains the historical sales data for a set of stores. It includes information on store sales, including variables related to the store, the sales itself, and specific features like promotions.

## Data Files

The dataset is split into the following files:

1. **train.csv** - Contains the historical sales data.
2. **test.csv** - Contains the test data that you will use to generate forecasts.
3. **stores.csv** - Contains metadata about the stores.
4. **oil.csv** - Contains oil prices data.
5. **holidays_events.csv** - Contains information about holidays and events.
6. **sample_submission.csv** - Contains a sample submission file that you will use as a template for your submission.

## File Descriptions

### `train.csv`

- **id**: Unique identifier for the row.
- **date**: The date of the record.
- **store_nbr**: The number of the store.
- **family**: The category of the product.
- **sales**: The amount of sales.
- **onpromotion**: The number of items on promotion.

### `test.csv`

- **id**: Unique identifier for the row.
- **date**: The date of the record.
- **store_nbr**: The number of the store.
- **family**: The category of the product.
- **onpromotion**: The number of items on promotion.

### `stores.csv`

- **store_nbr**: The number of the store.
- **city**: The city where the store is located.
- **state**: The state where the store is located.
- **type**: The type of store.
- **cluster**: A cluster identifier for grouping stores.

### `oil.csv`

- **date**: The date of the record.
- **dcoilwtico**: The West Texas Intermediate (WTI) crude oil price in USD.

### `holidays_events.csv`

- **date**: The date of the holiday/event.
- **type**: The type of holiday/event (e.g., public, religious, etc.).
- **locale**: The locale of the holiday/event (e.g., regional, local, etc.).
- **locale_name**: The name of the locale.
- **description**: Description of the holiday/event.

### `sample_submission.csv`

- **id**: Unique identifier for the row.
- **sales**: Placeholder for your forecasted sales values.

## Additional Notes

- **Data Granularity**: The data is provided at a daily granularity.
- **Promotions**: `onpromotion` indicates the number of items being promoted and can be zero.
- **Sales**: Sales data is given in units.

Remember to handle missing values and perform necessary preprocessing to make the data suitable for your forecasting models.


## Preprocessing the data

In [5]:
for i in dataframes:
    print(i.info(),'\n','-'*60)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB
None 
 ------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB
None 
 ------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     

In [6]:
for i in dataframes:
    print(i.name, '\n', i.isna().sum(),'\n','-'*60)

train 
 id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64 
 ------------------------------------------------------------
oil 
 date           0
dcoilwtico    43
dtype: int64 
 ------------------------------------------------------------
stores 
 store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64 
 ------------------------------------------------------------
transactions 
 date            0
store_nbr       0
transactions    0
dtype: int64 
 ------------------------------------------------------------
holidays 
 date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64 
 ------------------------------------------------------------


As we can see some of the date columns are not in datetime from pandas, also there are some missing values in oil

In [15]:
df_train['date'] = pd.to_datetime(df_train['date'])
df_oil['date'] = pd.to_datetime(df_oil['date'])
df_transactions['date'] = pd.to_datetime(df_transactions['date'])
df_holidays['date'] = pd.to_datetime(df_holidays['date'])

### treating null with moving avg based on month

In [16]:
df_oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        1218 non-null   datetime64[ns]
 1   dcoilwtico  1175 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 19.2 KB


In [18]:
df_oil.rename(columns = {'dcoilwtico':'oil_price'}, inplace=True)

In [38]:
df_m_avg = df_oil.groupby([df_oil['date'].dt.year, df_oil['date'].dt.month])['oil_price'].mean()

In [46]:
df_oil['date'].dt.year

0       2013
1       2013
2       2013
3       2013
4       2013
        ... 
1213    2017
1214    2017
1215    2017
1216    2017
1217    2017
Name: date, Length: 1218, dtype: int32

In [69]:
df_nan = df_oil[df_oil['oil_price'].isna()]
df_nan['y'] = df_nan['date'].dt.year
df_nan['m'] = df_nan['date'].dt.month
df_nan

C:\Users\T-Gamer\AppData\Local\Temp\ipykernel_26332\3949581012.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nan['y'] = df_nan['date'].dt.year
C:\Users\T-Gamer\AppData\Local\Temp\ipykernel_26332\3949581012.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nan['m'] = df_nan['date'].dt.month


,date,oil_price,y,m
0,2013-01-01,NaN,2013,1
14,2013-01-21,NaN,2013,1
34,2013-02-18,NaN,2013,2
63,2013-03-29,NaN,2013,3
104,2013-05-27,NaN,2013,5
132,2013-07-04,NaN,2013,7
174,2013-09-02,NaN,2013,9
237,2013-11-28,NaN,2013,11
256,2013-12-25,NaN,2013,12
261,2014-01-01,NaN,2014,1


In [71]:
df_m_avg.loc[2017, 7]

46.630526315789474

In [175]:
for y,m in zip(df_nan['y'], df_nan['m']):
    df_nan[(df_nan.m == m) & (df_nan.y == y)]['oil_price'] = df_m_avg.loc[y][m]

C:\Users\T-Gamer\AppData\Local\Temp\ipykernel_26332\3031890060.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nan[(df_nan.m == m) & (df_nan.y == y)]['oil_price'] = df_m_avg.loc[y][m]


In [170]:
df_nan[(df_nan.m == 1) & (df_nan.y == 2013)]['oil_price']

0    NaN
14   NaN
Name: oil_price, dtype: float64

In [171]:
df_m_avg[2013]

date
1      94.756667
2      95.308947
3      92.938500
4      92.021364
5      94.509545
6      95.772500
7     104.670909
8     106.572727
9     106.289500
10    100.538261
11     93.864000
12     97.625238
Name: oil_price, dtype: float64

In [113]:
df_m_avg.loc[2013][1]

94.75666666666667

In [96]:
df_m_avg.index[range(len(df_m_avg))][range(len(df_m_avg))]

MultiIndex([(2013,  1),
            (2013,  2),
            (2013,  3),
            (2013,  4),
            (2013,  5),
            (2013,  6),
            (2013,  7),
            (2013,  8),
            (2013,  9),
            (2013, 10),
            (2013, 11),
            (2013, 12),
            (2014,  1),
            (2014,  2),
            (2014,  3),
            (2014,  4),
            (2014,  5),
            (2014,  6),
            (2014,  7),
            (2014,  8),
            (2014,  9),
            (2014, 10),
            (2014, 11),
            (2014, 12),
            (2015,  1),
            (2015,  2),
            (2015,  3),
            (2015,  4),
            (2015,  5),
            (2015,  6),
            (2015,  7),
            (2015,  8),
            (2015,  9),
            (2015, 10),
            (2015, 11),
            (2015, 12),
            (2016,  1),
            (2016,  2),
            (2016,  3),
            (2016,  4),
            (2016,  5),
            (201

In [105]:
[m for m in df_nan['m']]

[1,
 1,
 2,
 3,
 5,
 7,
 9,
 11,
 12,
 1,
 1,
 2,
 4,
 5,
 7,
 9,
 11,
 12,
 1,
 1,
 2,
 4,
 5,
 7,
 9,
 11,
 12,
 1,
 1,
 2,
 3,
 5,
 7,
 9,
 11,
 12,
 1,
 1,
 2,
 4,
 5,
 7,
 7]

### treating null with interpolation

In [185]:
df_oil['oil_price'] = df_oil['oil_price'].interpolate().bfill()
df_oil.isna().sum()

date         0
oil_price    0
dtype: int64